# Proyecto 5: Análisis de sentimiento en reviews de películas con keras

Vamos a utilizar la biblioteca Keras de Deep Learning para realizar la misma clasificación que hicimos en el proyecto 5.

## Dataset

En primer lugar, carguemos las reviews para comenzar a procesarlas. En este caso no tenemos un archivo `.csv`, sino un directorio estructurado de la siguiente forma: 
```
movie_reviews/
  neg/
  pos/
```
`sklearn` nos provee de la función `load_files` que permite recorrer este tipo de estructuras. 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

Ahora en la variable `movies` tendremos guardadas las reviews (`movies.data`) y su sentimiento (`movies.target`).
Veamos cuantas reviews tenemos en total:

In [2]:
print("Tenemos {} reviews, con clases {},".format(len(movie_reviews.data), movie_reviews.target_names))

Tenemos 2000 reviews, con clases ['neg', 'pos'],


Analizando la estructura de los documentos, veamos como luce una muestra de `movies.data`:

In [3]:
movie_reviews.data[0][:500]

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so cal"

Y ahora, con la propiedad `target` podemos ver la categoría asociada a esta review

In [4]:
movie_reviews.target[0]

0

En este caso es un 0, es decir `neg`.

## Separando en conjuntos de entrenamiento y de testeo

Antes de comenzar con el entrenamiento de los modelos probando distintos algoritmos para encontrar los mejores modelos, vamos a separar el conjunto de reviews en training y testing. 

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

## Preprocesamiento

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vect = CountVectorizer(max_features=10000)

Es importante destacar que el `fit` debe hacerse sobre el conjunto de `train` y no sobre el total, ya que `CountVectorizer` tiene en cuenta los términos existentes, por lo que si usamos el conjunto de test podrían aparecer nuevos términos que no contemplamos. Una vez que usamos el `fit` con el conjunto de entrenamiento podemos aplicar la transformación al conjunto de `test`. 

In [8]:
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

### Dimensión de la matriz de términos
Una vez vectorizados los documentos veamos qué forma tiene la matriz resultante

In [9]:
X_train.shape
max_features = X_train.shape[1]

## Entrená una red neuronal con Keras


In [10]:
#Importar la librería Sequential y layers
from keras.models import Sequential
from keras import layers

Using TensorFlow backend.


In [11]:
# Constuir el modelo con 3 capas. 
# Qué función de activación eligirían para cada capa?

model = Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(max_features,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Elegir el batch_size
model.fit(X_train,y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 1280 samples, validate on 320 samples
Epoch 1/10
1280/1280 [==============================] - 1s 741us/step - loss: 0.7251 - acc: 0.5109 - val_loss: 0.6821 - val_acc: 0.5188
Epoch 2/10
1280/1280 [==============================] - 0s 140us/step - loss: 0.6415 - acc: 0.6477 - val_loss: 0.6339 - val_acc: 0.6594
Epoch 3/10
1280/1280 [==============================] - 0s 144us/step - loss: 0.5955 - acc: 0.6781 - val_loss: 0.6125 - val_acc: 0.6750
Epoch 4/10
1280/1280 [==============================] - 0s 148us/step - loss: 0.5766 - acc: 0.6891 - val_loss: 0.5834 - val_acc: 0.6938
Epoch 5/10
1280/1280 [==============================] - 0s 149us/step - loss: 0.4812 - acc: 0.7680 - val_loss: 0.6435 - val_acc: 0.6031
Epoch 6/10
1280/1280 [==============================] - 0s 147us/step - loss: 0.3947 - acc: 0.8563 - val_loss: 0.5990 - val_acc: 0.6562
Epoch 7/10
1280/1280 [==============================] - 0s 134us/step - loss: 0.3761 - ac

## Matriz de confusión 
Una forma de ver fácilmente el resultado de un clasificador es utilizando una matriz de confusión. A continuación
se presenta una función para visualizar una matriz de confusión utilizando `matplotlib`.

In [13]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict_classes(X_test)

from keras.utils import np_utils
y_pred_vec = np_utils.to_categorical(y_pred)

print(classification_report(y_test, y_pred_vec[:,1]))
print(accuracy_score(y_test, y_pred_vec[:,1]))

              precision    recall  f1-score   support

           0       0.73      0.95      0.83       200
           1       0.94      0.65      0.77       200

    accuracy                           0.80       400
   macro avg       0.83      0.80      0.80       400
weighted avg       0.83      0.80      0.80       400

0.8025


#### - Ejercicio: repetir modificando el número de muestras que se usa para entrenar


#### - DESAFÍO

Aplicación de redes neuronales para el precio de las casas
https://www.kaggle.com/diegosiebra/neural-network-model-for-house-prices-keras

* Qué función de pérdida usa?
* Qué función de activación se usa en la última capa? Por qué creen que hacen esto?
